In [2]:
import os
os.getcwd()


'c:\\Users\\Basti\\OneDrive\\Desktop\\Master\\NOVA SBE\\S2\\T1\\Adpro\\group project\\Group_13'

In [3]:
os.chdir(r"C:\Users\Basti\OneDrive\Desktop\Master\NOVA SBE\S2\T1\Adpro\group project")

In [4]:
import pandas as pd

# Define column names for df_movie
movie_columns = [
    "Wikipedia_movie_ID", 
    "Freebase_movie_ID", 
    "Movie_name", 
    "Movie_release_date", 
    "Box_office_revenue", 
    "Runtime", 
    "Languages", 
    "Countries", 
    "Genres"
]

# Define column names for df_character
character_columns = [
    "Wikipedia_movie_ID",
    "Freebase_movie_ID",
    "Movie_release_date",
    "Character_name",
    "Actor_date_of_birth",
    "Actor_gender",
    "Actor_height_meters",
    "Actor_ethnicity",
    "Actor_name",
    "Actor_age_at_release",
    "Freebase_character_actor_map_ID",
    "Freebase_character_ID",
    "Freebase_actor_ID"
]

# Read both datasets with proper column names
df_movie = pd.read_csv("movie.metadata.tsv", delimiter="\t", header=None, names=movie_columns)
df_character = pd.read_csv("character.metadata.tsv", delimiter="\t", header=None, names=character_columns)

# Display the first few rows of each DataFrame to confirm changes
print("Movies DataFrame:")
display(df_movie.head())

print("Characters DataFrame:")
display(df_character.head())



Movies DataFrame:


,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Box_office_revenue,Runtime,Languages,Countries,Genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


Characters DataFrame:


,Wikipedia_movie_ID,Freebase_movie_ID,Movie_release_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height_meters,Actor_ethnicity,Actor_name,Actor_age_at_release,Freebase_character_actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


You are going to build a **[Streamlit app](https://streamlit.io/)** that will showcase your analysis.  
Keep all the .py and .ipynb out of the main directory. The only files in the main directory of the project should be the **files necessary to launch the app** and the several configuration files (.yml, .gitignore, and others). Everything else should have their own directories, like downloaded content.
